<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/Fam_Prom_PowerBI_Project_90_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.10.0'

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/drive/My Drive/FPData/90 Day Data/FP_90_Day_Mar_25.xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [0]:
#currently_enrolled.head()

In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more

from datetime import date, timedelta

current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [0]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [0]:
#exit_df.head()

In [0]:
exit_df.to_excel('/content/drive/My Drive/FPData/90 Day Data/90Day_exit_df.xlsx')

In [29]:
households = df['5.9 Household ID'].unique()
len(households)

86

In [30]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

86

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [32]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

39.32

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [0]:
#df.head()

In [0]:
perm_leaver = df[df['perm_leaver'] == True]

In [0]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [39]:
total_perm

57

In [0]:
non_perm = df[df['perm_leaver'] == False]

In [0]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [42]:
total_non_perm

73

In [0]:
perm_percent = total_perm / total_non_perm

In [44]:
perm_percent = round(perm_percent, 2)*100
perm_percent

78.0

In [45]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             53
Rental by client, no ongoing housing subsidy                                                                                     30
Rental by client, other ongoing housing subsidy                                                                                  24
Transitional Housing for homeless persons (including homeless youth)                                                              6
Staying or living with family, temporary tenure (e.g., room, apartment or house)                                                  5
Client refused                                                                                                                    4
Rental by client with RRH or equivalent subsidy                                                                                   3
Substance Abuse Treatment or Detox Center                                   

In [0]:
unknowns = ['Client refused', 'No exit interview completed']

In [0]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [0]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [49]:
unknown_exits

4

In [50]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

5.0

In [51]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

130

In [0]:
exits = df['3.12 Exit Destination'].value_counts()

In [53]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] 
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))
esExits
#print('Exits to Transitional',transExits,'%')

53

In [54]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()

tempExits

5

In [55]:
transitional = df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)'
transitional = transitional.sum()
transitional

6

In [56]:
exits.sum()

130

In [0]:
total_guests = total_non_perm + total_perm

In [58]:
trans_percent = (transitional / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

4.62

In [0]:
#zips_master

In [60]:
# export zips into an excel sheet for Power BI visual
zips_master = pd.read_excel('/content/drive/My Drive/FPData/uszips.xlsx')



# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
zips_bi = df['V5 Zip'].value_counts().to_frame('Total').reset_index(level=0, inplace=False)
zips_bi = zips_bi.rename(columns={"index": "V5 Zip"})
zips_bi = zips_bi.merge(zips_master,on='V5 Zip')
zips_bi['V5 City'].value_counts()

Spokane           7
Spokane Valley    2
Kennewick         1
Airway Heights    1
Hemet             1
Nampa             1
Lolo              1
Tacoma            1
Mountain Home     1
Omak              1
Browning          1
Sprague           1
Deer Park         1
Tillamook         1
Cheney            1
Name: V5 City, dtype: int64

In [61]:
type(zips_bi)

pandas.core.frame.DataFrame

In [0]:
zips_bi.to_excel('/content/drive/My Drive/FPData/90 Day Data/weekly_90Day_zips.xlsx', header=True, index=False)

In [0]:
other_reasons = ['Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)',
                 'Hotel or Motel paid for without Emergency Shelter Voucher', 'Other', 'Substance Abuse Treatment or Detox Center',
                 'Foster Care Home or Foster Care Group Home', 'Psychiatric Hospital or Other Psychiatric Facility']

In [0]:
other = df['3.12 Exit Destination'].isin(other_reasons)

In [65]:
other.sum()
other = other.sum()
other

5

In [66]:
df['4.04.7 Private - Employer']

0       No
1       No
2       No
3       No
4       No
      ... 
202    NaN
218     No
219     No
220     No
221     No
Name: 4.04.7 Private - Employer, Length: 130, dtype: object

In [67]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             53
Rental by client, no ongoing housing subsidy                                                                                     30
Rental by client, other ongoing housing subsidy                                                                                  24
Transitional Housing for homeless persons (including homeless youth)                                                              6
Staying or living with family, temporary tenure (e.g., room, apartment or house)                                                  5
Client refused                                                                                                                    4
Rental by client with RRH or equivalent subsidy                                                                                   3
Substance Abuse Treatment or Detox Center                                   

In [68]:
print(total_guests, 'total guests served YTD')
print(total_non_perm, 'total non perm leavers')
print(total_perm, 'total perm leavers')
print(esExits, 'total ES leavers')
print(tempExits, 'total temp leavers')
print(transitional, 'total trans leavers')
print(other, 'total leavers to "other"')
print(unknown_exits, 'total unknown leavers')
print('sum of all groups to make sure it matches total served:',total_perm+esExits+tempExits+transitional+other+unknown_exits)

130 total guests served YTD
73 total non perm leavers
57 total perm leavers
53 total ES leavers
5 total temp leavers
6 total trans leavers
5 total leavers to "other"
4 total unknown leavers
sum of all groups to make sure it matches total served: 130


In [0]:
assert((total_perm+esExits+tempExits+transitional+other+unknown_exits) == (total_non_perm+total_perm))

In [0]:
perm_bi = '{} %'.format(perm_percent)
es_bi = '{} %'.format(esExits)
temp_bi = '{} %'.format(tempExits)
trans_bi = '{} %'.format(trans_percent)
unknown_bi = '{} %'.format(unknown_percent)

In [0]:
power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, 
                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, 
                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])


In [0]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', transitional],
        ['Unknown Leavers', unknown_exits], ['Other Leavers', other]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [73]:
power_bi

,Exit Destination,Total
0,Permanent Leavers,57
1,Emergency Shelter Leavers,53
2,Temporary Leavers,5
3,Transitional Leavers,6
4,Unknown Leavers,4
5,Other Leavers,5


In [74]:
power_bi.Total.sum()

130

In [0]:
power_bi.to_excel('/content/drive/My Drive/FPData/90 Day Data/90Day_weeklyDataNew.xlsx')